In [1]:
#!pip install --quiet glvis
import numpy as np
import matplotlib.pyplot as plt
from  bfp import *
import mfem.ser as mfem
#from mfem.common.arg_parser import ArgParser
from glvis import glvis, GlvisData
import seaborn as sns
import pandas as pd
import os

In [7]:
nx = 10
nE = 10
x_start = 0.0
x_end = 1.0
E_start = 0.0
E_end = 1.0
N_ang = 2
order = 1
inflow = 10.0

mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
#mesh = mfem.Mesh(nx, nE, "QUADRILATERAL", True, x_end, E_end)
mesh.UniformRefinement()
dim = mesh.Dimension()

# btype = 0-GaussLegendre, 1 - GaussLobatto
fec = mfem.DG_FECollection(order, dim, btype=0)
fes = mfem.FiniteElementSpace(mesh, fec)
Size = fes.GetVSize()
print("Number of unknowns:", Size)
print('Number of finite element unknowns: ' + str(fes.GetVSize()))

mu_vals, w_vals = gauss_legendre_dirs(N_ang)

#ex17'den


ess_tdof_list1 = mfem.intArray()
dir_bdr1 = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr1.Assign(0)
dir_bdr1[3] = 1
dir_bdr1[1] = 1

ess_tdof_list2 = mfem.intArray()
dir_bdr2 = mfem.intArray(mesh.bdr_attributes.Max())
dir_bdr2.Assign(0)
dir_bdr2[0] = 1
dir_bdr2[2] = 1

S_const = 0.0
xs_t_const = 5.0
#xs_s_const = 1.0
S_arr = np.ones(nE) * S_const
E_arr = np.linspace(E_start, E_end, nE+1)

S_coeff = StoppingPowerCoefficientE(S_const, E_start, E_end)
xs_t_coeff = TotalXSCoefficientE(xs_t_const, E_start, E_end)
#xs_s_coeff = EDependentCoefficient(xs_s_const, E_start, E_end)
q_coeff = QCoefficientE(0.0, E_start, E_end)

a = 5
b = 10
'''
class QCoefficient(mfem.PyCoefficient):
    def __init__(self):
        mfem.PyCoefficient.__init__(self)
    def EvalValue(self, x):
        return (2*10+5*(5+10*x[1]))
q_coeff = QCoefficient()
'''
# Solution

alpha = 1.0
beta = 0.5

psi_mu = []
psi_mu_list = []

for mu, w in zip(mu_vals, w_vals):
    print("  Solving for mu =", mu)
    mu_coeff = ConstantCoefficient(mu)
    inflow_coeff = InflowCoefficient(inflow, mu)
    #marker = get_marker_for_mu(mesh, mu)
    #v_coeff = MyVectorCoefficient(mu_coeff, S_coeff)
    v_coeff1 = mfem.VectorConstantCoefficient([mu, 0.0])
    v_coeff2 = mfem.VectorConstantCoefficient([0.0, S_const])
    print(f"inflow: {inflow_coeff.EvalValue([inflow, mu])}")
    #print(f"v_coeff: {v_coeff.EvalValue([mu, 0.8])}")
    print(f"sigma_t: {xs_t_coeff.EvalValue([0.0, 0.604])}")


    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff1, 1.0))
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff2, 1.0))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff1, -alpha, beta)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff1, -alpha, beta)), dir_bdr1)
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff2, -alpha, beta)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff2, -alpha, beta)), dir_bdr2)
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff1, -alpha, -beta), dir_bdr1)
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff2, -alpha, -beta), dir_bdr2)
    b.Assemble()
    
    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)

    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()
    solver.SetOperator(A)
    solver.SetPreconditioner(prec)
    solver.SetRelTol(1e-12)
    solver.SetAbsTol(1e-12)
    solver.SetMaxIter(1000)
    solver.SetKDim(30)
    solver.SetPrintLevel(1)
    solver.Mult(b, psi)
    

    g = glvis((mesh, psi), 500,500, keys='ARjmcbp')
    g.set_size(1000, 1000)
    g.render()
    
    #X_arr = mfem.GridFunction(fes)
    #X_arr.Assign(X)
    psi.Save("psi_hs_mu_{:.3f}.gf".format(mu))
    psi_mu.append(psi.GetDataArray())
    psi_mu_list.append((mu, w, psi))


phi = mfem.GridFunction(fes)
phi.Assign(0.0)
for mu, w, X in psi_mu_list:
    phi.Add(w, X)

phi.Save("phi_dg.gf")

    

File '/Users/melekderman/github/BFP/melek/mesh/usr/10x10_2D.mesh' already exists.
Number of unknowns: 1600
Number of finite element unknowns: 1600
  Solving for mu = -0.5773502691896257
inflow: 10.0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 156.62
   Pass :  1   Iteration :   1  ||B r|| = 109.808
   Pass :  1   Iteration :   2  ||B r|| = 78.2367
   Pass :  1   Iteration :   3  ||B r|| = 58.9902
   Pass :  1   Iteration :   4  ||B r|| = 49.3477
   Pass :  1   Iteration :   5  ||B r|| = 39.4615
   Pass :  1   Iteration :   6  ||B r|| = 32.1731
   Pass :  1   Iteration :   7  ||B r|| = 25.3825
   Pass :  1   Iteration :   8  ||B r|| = 19.1648
   Pass :  1   Iteration :   9  ||B r|| = 14.3244
   Pass :  1   Iteration :  10  ||B r|| = 10.7171
   Pass :  1   Iteration :  11  ||B r|| = 8.37513
   Pass :  1   Iteration :  12  ||B r|| = 6.08642
   Pass :  1   Iteration :  13  ||B r|| = 4.4253
   Pass :  1   Iteration :  14  ||B r|| = 3.216
   Pass :  1   Iteration :  15  ||B r|| = 

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

  Solving for mu = 0.5773502691896257
inflow: 10.0
sigma_t: 5.0
   Pass :  1   Iteration :   0  ||B r|| = 123.585
   Pass :  1   Iteration :   1  ||B r|| = 92.7559
   Pass :  1   Iteration :   2  ||B r|| = 68.2242
   Pass :  1   Iteration :   3  ||B r|| = 52.7934
   Pass :  1   Iteration :   4  ||B r|| = 42.6317
   Pass :  1   Iteration :   5  ||B r|| = 33.2858
   Pass :  1   Iteration :   6  ||B r|| = 25.5342
   Pass :  1   Iteration :   7  ||B r|| = 18.8633
   Pass :  1   Iteration :   8  ||B r|| = 13.9891
   Pass :  1   Iteration :   9  ||B r|| = 10.5387
   Pass :  1   Iteration :  10  ||B r|| = 7.88168
   Pass :  1   Iteration :  11  ||B r|| = 5.82085
   Pass :  1   Iteration :  12  ||B r|| = 4.10868
   Pass :  1   Iteration :  13  ||B r|| = 3.10681
   Pass :  1   Iteration :  14  ||B r|| = 2.25639
   Pass :  1   Iteration :  15  ||B r|| = 1.66172
   Pass :  1   Iteration :  16  ||B r|| = 0.891886
   Pass :  1   Iteration :  17  ||B r|| = 0.561108
   Pass :  1   Iteration :  18  ||

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [8]:
g = glvis((mesh, phi), 500,500, keys='ARjmcbp')
g.set_size(900, 900)
g.render()

_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [9]:
g = glvis((mesh, phi), 500,500, keys='rljgac//0')
g.set_size(900, 900)
g.render()


_GlvisWidgetCore(data_str='solution\nMFEM mesh v1.0\n\n#\n# MFEM Geometry Types (see fem/geom.hpp):\n#\n# POIN…

In [5]:
E = np.linspace(0,1,11)
for i in range(E.shape[0]):
    print(((5+10*i)))

5
15
25
35
45
55
65
75
85
95
105
